### Berechnung Angebot und Nachfrage nach Milch und Milchprodukten

Nachfragemodellierung
- Gesamtbevölkerung 84,37 Mil: https://de.statista.com/statistik/daten/studie/2861/umfrage/entwicklung-der-gesamtbevoelkerung-deutschlands/
- BEvölkerung Sachsen 4,09 Millionen Einwohner: https://de.statista.com/statistik/daten/studie/155167/umfrage/entwicklung-der-bevoelkerung-von-sachsen-seit-1961/

Quelle: https://milchindustrie.de/wp-content/uploads/2023/09/ZahlenDatenFakten_2023_neu.pdf

| | Milchpulver | Konsummilch | Butter | Käse |
| --- | --- | --- | --- | --- |
| Herstellungsmenge (T Tonnen) | 630 | 7300 | 480 | 2670 |

In [7]:
anteil = 4.09 / 84.37
anteil_jahr = 30 / 360
products = [630, 7300, 480, 2670]
products_right = [int(round(prod * 1000 * anteil * anteil_jahr,0)) for prod in products]

milchanlieferung = round((31021.6 * 1000) * (30/360) * anteil) #Quelle https://milchindustrie.de/wp-content/uploads/2023/09/ZahlenDatenFakten_2023_neu.pdf
# milchanlieferung = round((1.7 * 10**6) * (30/360)) #1,7 Milliarden kg  Sachsenmilch Webiste jährlich prod menge 

products_right.append(milchanlieferung)


# Gegebene Liste mit Sublisten
original_demand = [
    [6400, 8000, 12000],
    [600, 1000, 1200],
    [2800, 4000, 8000],
    [300, 500, 600],
    [862.5, 1725, 3450]
]

# Berechnung der prozentualen Schwankungen
percent_changes = []

for sublist in original_demand:
    original_value = sublist[1]  # Das mittlere Produkt entspricht dem Original
    changes = []
    for value in sublist:
        percent_change = ((value - original_value) / original_value) * 100
        changes.append(percent_change)
    percent_changes.append(changes)

new_values = []

for p in range(len(products_right)):
    sublist = []
    for change in percent_changes[p]:
        sublist.append(round(products_right[p] * (1 + change / 100)))

    new_values.append(sublist)

new_values

[[2036, 2545, 3818],
 [17694, 29490, 35388],
 [1357, 1939, 3878],
 [6472, 10786, 12943],
 [62660, 125319, 250638]]

### Festlegung: 

- 4 Produkltfamilien(Powdered Milk, Milk, Butter, Cheese)
- 8 Produkte (Laktose, Magermilchpulver, 1.5 % Milch UHT, 3.5 % Milch UHT, Süßrahmbutter, Butter mild gesäuert, Gouda, Emmentaler)
- 2 Fresh and 2 Dry! 
- Jede Produltfamilie besitzt eigene Fertigungsfläche und Lagerung dort 
- 3 Distribution Centers 
- Ein Monat Planungszeitraum 

"T_No": 30,
"F_No": 4,
"FT_No": 2,
"MP_No": 4,
"CT_No": 2,
"L_No": 4,
"P_No": 8,

### Annahme rr

es werden die gleichen verhältnisse, wie im paper angenommen:


In [13]:
rr_old = [3.75,1, 0, 0]
r_old = [12.5, 3.425, 5, 12]

verhältnisse = [rr_old[i]/r_old[i] for i in range(len(rr_old))]

r_new = [3165, 464, 6460, 5063]
rr_new = [round(r_new[i] * verhältnisse[i]) for i in range(len(r_new))]

rr_new

[950, 135, 0, 0]

### Kalkulation Produktionskapazitäten

Annahme, dass Molkereibetrieb alle nachgefagten Mengen im Base Case prodzieren könnte und noch 30% mehr! Bis auf milchpulver produktion 3 Schichten Verfügbar! 

In [22]:
produktion_monthly = products_right[:4]

cmax_new = [round(prod*(1/30)*1.30) for prod in produktion_monthly]

sc_new = [0] + [round(prod/3) for prod in cmax_new[1:]]

print("Cmax_new: ", cmax_new)
print("Sc_new: ", sc_new)

Sc_new:  [0, 426, 28, 156]
Cmax_new:  [110, 1278, 84, 467]


### Überarbeitete Dateneingabe

```python
{
    "T_No": 30,
    "F_No": 4,
    "FT_No": 2,
    "MP_No": 4,
    "CT_No": 2,
    "L_No": 3,
    "fty": [0,0,1,1], #Stays the same 
    "cty": [0,1,1,1], #Stays the same 
    "fpr": [0,1,2,3], #Stays the same 
    "fy": [0.12,0.975, 0.25, 0.11], #Adapted to butter value
    "rsc": 451.6, # Aus Paper
    "roc": 500, #Assumption that costs are increasing
    "el": [24,24,24,24], 
    "tau": [1,1,1,1], 
    "i_0": [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
    "i_0_f": [0,0,0,0],
    "tl_min": 12,
    "tl_max": 24,
    "r0": 0, 
    "r_max": 2000,
    "dmax": [15,100,100,100],
    "cmin": [1, 1, 1, 1],
    "cmax": [110, 1278, 84, 467], #Adapted to given values
    "alpha": [2,0,0,0],
    "ost": [0, 0, 0, 0],
    "wp": [[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [20, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[10,20,15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    "el_min": [0,0,0,0],
    "el_max": [1000,1000,1000,1000],
    "is": [0, 0.95, 0.95, 0.95], #Stays the same 
    "omega_fw": [40, 40, 5, 10], #Stays the same 
    "omega_dc": [100, 100, 7, 14], #Stays the same 
    "rr": [950, 135, 0, 0], # Calculated from all the other values cumulated
    "r": [3165, 464, 6460, 5063], # Own Calculation
    "re": [3115, 414, 6410, 5013], # Own assumption
    "imax": [[1000,1000],[1000,1000],[1000,1000],[1000,1000]] # Constant values for all DCs
    "zmax": [1,3,3,3], #Stays the same 
    "sc": [0, 426, 28, 156], #Adapted to given values
    "beta": [0.1,0,0,0], #Stays the same 
    "sigma": [1,0,0,4], #Stays the same 
    "iwip0": [0, 0, 0, 0],
    "tc": [[242,213], [178,157], [270,237], [39,39]], #Own Calculation 
    "sco": 20000, #Own assumption
    "K": 1,
    "epsilon": 0.001,
    "N": 10000,
    "demand_supply": [[2036, 2545, 3818], [17694, 29490, 35388], [1357, 1939, 3878], [6472, 10786, 12943], [62660, 125319, 250638]],
    "probabilies": [[0.4, 0.35, 0.25], [0.25, 0.35, 0.4], [0.4, 0.3, 0.3], [0.3, 0.4, 0.3], [0.5, 0.15, 0.35]]
}


In [14]:
125319 / 30

4177.3

### Export prices

In [9]:
[element -50 for element in [3165, 464, 6460, 5063]]

[3115, 414, 6410, 5013]

In [23]:
[element -50 for element in [2480, 3850,460, 468.5,6320, 6600, 4175, 5950]]

[2430, 3800, 410, 418.5, 6270, 6550, 4125, 5900]